In [2]:
import os
import httpx
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import numpy as np
import import_ipynb
import ollama
from evaluate import load
from rouge import Rouge


In [3]:
import import_ipynb
import importlib
import Eval_Metrics
eval_metrics = Eval_Metrics.Evaluation_Metrics()
# importlib.reload(Eval_Metrics)
# eval_metrics = Eval_Metrics.Evaluation_Metrics()

#### Chosen LLM

In [1]:
model = "llama3.1:latest"

### Getting Ranking List Prompt Dataset to be used in Ranking List LLM Evaluation

In [15]:
prompt_examples_df = pd.read_excel(r"C:\Users\flame\CP5105-AI-Powered-Product-and-Service-Review-Summarizer-and-Ranking-System\BackEnd\All Prompt Examples\Ranking List Prompt Dataset.xlsx")


In [16]:
prompt_examples_df['Modified_Multiple_Generated_Summaries'] = prompt_examples_df['Modified_Multiple_Generated_Summaries'].map(lambda a: a.replace('\n',' '))

In [17]:
prompt_examples_df.columns

Index(['Query', 'Modified_1_Generated_Summary',
       'Modified_Multiple_Generated_Summaries', 'Model_Ranking_List_Output',
       'Model_Ranking_List_for_Manual_Checking', 'Prompt', 'Unnamed: 6'],
      dtype='object')

#### Getting Data to be used as Prompt Examples from dataset

In [18]:
promptengin_df = prompt_examples_df[prompt_examples_df['Prompt']=='Use for Prompt'].reset_index(drop=True)
promptengin_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt,Unnamed: 6
0,What are the worst mobile phone games released...,The gaming industry has been flooded with new ...,Article #1: The gaming industry has been flood...,Here is the ranking list for the worst mobile ...,1: Robot Rebellion\n2. Galactic Conquest\nRest...,Use for Prompt,NaN
1,What are the top 5 best sci-fi movies of all t...,This is a question that has been debated by fi...,Article #1: This is a question that has been ...,Here is the ranking list for the top 5 best sc...,1: Blade Runner\n2: The Matrix\n3: A Space Ody...,Use for Prompt,NaN
2,What are the 3 worst games consoles of all time?,The Atari Jaguar is often cited as one of the ...,Article #1: The Atari Jaguar is often cited a...,Here is the ranking list for the 3 worst games...,"1,2: Atari Jaguar, Nintendo Virtual Boy\n3: Se...",Use for Prompt,NaN
3,What are the 5 worst board games?,Determining the worst board games can be subje...,Article #1: Determining the worst board games ...,Here is the ranking list for the 5 worst board...,1: Monikers\n2: Don’t Wake Daddy\n3: Battleshi...,Use for Prompt,NaN
4,What are the best printer models in 2024?,Choosing the right printer can be overwhelming...,Article #1: Choosing the right printer can be ...,Here is the ranking list for the best printer ...,1. Epson Expression Photo HD XP-15000\n2. Xero...,Use for Prompt,NaN


#### Getting Data to be used for Ranking List LLM testing from dataset

In [19]:
testprompt_df = prompt_examples_df[prompt_examples_df['Prompt'].isna()==True].reset_index(drop=True)
testprompt_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt,Unnamed: 6
0,What are the top car models in 2024?,The automotive industry is constantly evolving...,Article #1: The automotive industry is constan...,Here is the ranking list for the top car model...,"1,2: Toyota Camry , Honda Civic Type R\n3,4: H...",NaN,What are the top car models in 2024?
1,What are the 3 worst laptop brands in 2024?,The ranking of laptop brands can vary dependin...,Article #1: The ranking of laptop brands can v...,Here is the ranking list for the 3 worst lapto...,1: Acer\n2: Lenovo\n3: Asus,NaN,What are the 3 worst laptop brands in 2024?
2,What are the cheapest but decent laptop models...,"When it comes to budget-friendly laptops, ther...",Article #1: When it comes to budget-friendly l...,Here is the ranking list for the top budget la...,"1: Lenovo IdeaPad 330S\n2,3, 4 ,5: Acer Aspire...",NaN,What are the cheapest but decent laptop models...
3,What are the top sunscreen brands?,Sunscreen is a vital part of our daily skincar...,Article #1: Sunscreen is a vital part of our ...,Here is the ranking list for the top sunscreen...,"1. Neutrogena\n2. Hawaiian Tropic\n3,4,5: La R...",NaN,What are the top sunscreen brands?
4,What are the most interesting and top rated bo...,The year 2024 is expected to bring a diverse r...,Article #1: The year 2024 is expected to bring...,Here is the ranking list for the most interest...,"1,2: The It Girl, The Memory Keeper\'s Daughte...",NaN,What are the most interesting and top rated bo...
5,What are the most hated and worst books releas...,"Based on user ratings on Goodreads, some of th...",Article #1: Based on user ratings on Goodreads...,Here is the ranking list for the most hated an...,"1,2: No Going Back , Crown of Starlight\n3,4 :...",NaN,What are the most hated and worst books releas...
6,What are the 5 worst horror movies of all time?,The assessment of the worst horror movies is s...,Article #1: The assessment of the worst horror...,Here is the ranking list for the 5 worst horro...,1: Devil\n2: Birdemic: Shock and Terror (2010)...,NaN,What are the 5 worst horror movies of all time?
7,What are the 3 best games consoles of all time?,"Determining the ""best"" gaming consoles is subj...","Article #1: Determining the ""best"" gaming cons...",Here is the ranking list for the top 3 best ga...,"1,2: PlayStation 2 (PS2) , Nintendo Entertainm...",NaN,What are the 3 best games consoles of all time?
8,What are the 5 best board games for children?,Choosing the right board game can be a challen...,Article #1: Choosing the right board game can ...,Here is the ranking list for the 5 best board ...,"1,2,3,4 : Candy Land, Chutes and Ladders, Mono...",NaN,What are the 5 best board games for children?
9,What are the worst mobile phones in 2024?,The ranking of the worst mobile phones can be ...,Article #1: The ranking of the worst mobile ph...,Here is the ranking list for the worst mobile ...,"1. Motorola Defy Pro X\n2,3: HTC One M7, Samsu...",NaN,What are the worst mobile phones in 2024?


#### Ranking List response generation function for Zero Shot COT Default (0.8 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [9]:
def ZERO_STEP_SYSTEM_COT_SINGLE_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt + f'''
            Let\'s think step by step.
            1) Identify the item names relevant to the {query} in the user prompt text.
            2) Calculate number of times each item name is mentioned in the user prompt text.
            3) Create an ordered ranking list ordered from item name that appeared most amount of times in the user prompt text to the item name that appeared least amount of times in the user prompt text.
            4) Provide a description for each item name in the ordered ranking list.
            '''
        },
        {
            'role': 'user',
            'content': f'''In the context of "{query}" , generate an ordered and ranked list from the following text : '''
            + text,
        },

        ]
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot Default (0.8 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [4]:
def SINGLE_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },

        ]
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.96 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [11]:
def SINGLE_0_96T_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "temperature": 0.96
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.64 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [12]:
def SINGLE_0_64T_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "temperature": 0.64
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.9 Top P , 48 Top K ) SYS with SCT text input

In [13]:
def SINGLE_48topk_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_k": 48
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.9 Top P , 32 Top K ) SYS with SCT text input

In [14]:
def SINGLE_32topk_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_k": 32
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.9 Top P , Variable Top K ) SYS with SCT text input

In [15]:
def SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_k": Top_K
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , Variable Top P , 40 Top K ) SYS with SCT text input

In [16]:
def SINGLE_OPTIMIZATION_Top_P_Change_generate_report_with_query(text, model,query,prompt,Top_P):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": Top_P
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , Variable Top P , Variable Top K ) SYS with SCT text input

In [20]:
def SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": Top_P,
            "top_k": Top_K
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 1.08 Top P , 40 Top K ) SYS with SCT text input

In [18]:
def SINGLE_1_08topp_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": 1.08
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.72 Top P , 40 Top K ) SYS with SCT text input

In [19]:
def SINGLE_0_72topp_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": 0.72
        }
        )
    
    
    report = response['message']['content']
    
    return report

##### Response DataFrame Initialization

In [20]:
zero_shot_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','avg_score'])

In [ ]:
query =''

##### Chosen LLM

In [21]:
model = "llama3.1:latest"

### Evaluation

### Zero Shot Prompt

In [22]:
query =''

In [ ]:
def zero_shot_prompt_SINGLE(query):
        zero_shot_prompt =  f'''You are a reviewer generating an ordered and numbered ranked list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times in the user prompt text to the item that appeared least amount of times in the user prompt text. \
                Each item in the ordered list should have the item name and the item details from the user prompt text. \
                Indicate the appearance frequency for each item in the user prompt text.\
                Bold each item on the ranking list in the response.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                '''
        return zero_shot_prompt

#### Step Wise Regression

#### For Top K = 28

In [17]:
zero_shot_single_response_df_28Top_K = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 28
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_28Top_K.loc[-1] = ref_row
    zero_shot_single_response_df_28Top_K.index = zero_shot_single_response_df_28Top_K.index + 1  #shift index
    zero_shot_single_response_df_28Top_K = zero_shot_single_response_df_28Top_K.sort_index()  #sort by index

In [18]:
zero_shot_single_response_df_28Top_K.to_csv('zero_shot_single_response_df_28Top_K.csv')

In [ ]:
zero_shot_single_response_df_28Top_K.to_excel('zero_shot_single_response_df_28TopK_0_9TopP.xlsx')

#### For Top K = 36

In [19]:
zero_shot_single_response_df_36Top_K = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 36
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_36Top_K.loc[-1] = ref_row
    zero_shot_single_response_df_36Top_K.index = zero_shot_single_response_df_36Top_K.index + 1  #shift index
    zero_shot_single_response_df_36Top_K = zero_shot_single_response_df_36Top_K.sort_index()  #sort by index

In [20]:
zero_shot_single_response_df_36Top_K.to_csv('zero_shot_single_response_df_36Top_K.csv')

In [ ]:
zero_shot_single_response_df_36Top_K.to_excel('zero_shot_single_response_df_36TopK_0_9TopP.xlsx')

#### For Top K = 32

In [16]:
zero_shot_single_response_df_32Top_K = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 32
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_32Top_K.loc[-1] = ref_row
    zero_shot_single_response_df_32Top_K.index = zero_shot_single_response_df_32Top_K.index + 1  #shift index
    zero_shot_single_response_df_32Top_K = zero_shot_single_response_df_32Top_K.sort_index()  #sort by index

In [17]:
zero_shot_single_response_df_32Top_K.to_csv('zero_shot_single_response_df_32Top_K.csv')

In [ ]:
zero_shot_single_response_df_32Top_K.to_excel('zero_shot_single_response_df_32TopK_0_9TopP.xlsx')

#### For Top K = 38

In [17]:
zero_shot_single_response_df_38Top_K = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 38
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_K_Change_generate_report_with_query(text, model,query,prompt,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_38Top_K.loc[-1] = ref_row
    zero_shot_single_response_df_38Top_K.index = zero_shot_single_response_df_38Top_K.index + 1  #shift index
    zero_shot_single_response_df_38Top_K = zero_shot_single_response_df_38Top_K.sort_index()  #sort by index

In [18]:
zero_shot_single_response_df_38Top_K.to_csv('zero_shot_single_response_df_38Top_K.csv')

In [ ]:
zero_shot_single_response_df_38Top_K.to_csv('zero_shot_single_response_df_38Top_K.csv')

#### For Default

In [ ]:
zero_shot_single_response_df_DEFAULT = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_4 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_5 = SINGLE_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_DEFAULT.loc[-1] = ref_row
    zero_shot_single_response_df_DEFAULT.index = zero_shot_single_response_df_DEFAULT.index + 1  #shift index
    zero_shot_single_response_df_DEFAULT = zero_shot_single_response_df_DEFAULT.sort_index()  #sort by index

In [ ]:
zero_shot_single_response_df_DEFAULT.to_excel('zero_shot_single_response_df_DEFAULT_40TopK_0_9TopP.xlsx')

#### For Top K = 36 , Top P = 0.81

In [26]:
zero_shot_single_response_df_36Top_K_0_81Top_P = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 36
    Top_P = 0.81
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_36Top_K_0_81Top_P.loc[-1] = ref_row
    zero_shot_single_response_df_36Top_K_0_81Top_P.index = zero_shot_single_response_df_36Top_K_0_81Top_P.index + 1  #shift index
    zero_shot_single_response_df_36Top_K_0_81Top_P = zero_shot_single_response_df_36Top_K_0_81Top_P.sort_index()  #sort by index

In [27]:
zero_shot_single_response_df_36Top_K_0_81Top_P.to_excel('FINAL_zero_shot_single_response_df_36Top_K_0_81Top_P.xlsx')

In [ ]:
zero_shot_single_response_df_36Top_K_0_81Top_P.to_excel('zero_shot_single_response_df_36TopK_0_81TopP.xlsx')

#### For Top K = 36 , Top P = 0.72

In [21]:
zero_shot_single_response_df_36Top_K_0_72Top_P = pd.DataFrame(columns=['query',
                                                     'model_response',
                                                     'zero_shot_response_1',
                                                     'zero_shot_response_1_score',
                                                     'zero_shot_response_2',
                                                     'zero_shot_response_2_score',
                                                     'zero_shot_response_3',
                                                     'zero_shot_response_3_score',
                                                     'zero_shot_response_4',
                                                     'zero_shot_response_4_score',
                                                     'zero_shot_response_5',
                                                     'zero_shot_response_5_score',
                                                     'avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    zero_shot_response_4_score = ''
    zero_shot_response_5_score = ''
    avg_score = ''
    Top_K = 36
    Top_P = 0.72
    
    response_text_1 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_2 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_3 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_4 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)
    response_text_5 = SINGLE_OPTIMIZATION_Top_PANDK_Change_generate_report_with_query(text, model,query,prompt,Top_P,Top_K)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               response_text_4, 
               zero_shot_response_4_score, 
               response_text_5,
               zero_shot_response_5_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df_36Top_K_0_72Top_P.loc[-1] = ref_row
    zero_shot_single_response_df_36Top_K_0_72Top_P.index = zero_shot_single_response_df_36Top_K_0_72Top_P.index + 1  #shift index
    zero_shot_single_response_df_36Top_K_0_72Top_P = zero_shot_single_response_df_36Top_K_0_72Top_P.sort_index()  #sort by index

In [22]:
zero_shot_single_response_df_36Top_K_0_72Top_P.to_excel('FINAL_zero_shot_single_response_df_36Top_K_0_72Top_P.xlsx')